В зависимости от того что покупает человек, получится ли ему продать что-то дополнительно

In [2]:
import numpy as np
import pandas as pd

In [86]:
df = pd.DataFrame(data = {'type': ['букет', 'композиция', 'один цветок', 'композиция', 'один цветок',
                                   'несколько цветков', 'букет', 'композиция', 'один цветок'],
                          'price': [3000, 1500, 200, 2000, 300, 1500, 5000, 2000, 200],
                          'addition': [1, 0, 1, 1, 0, 1, 0, 1, 0]})

In [87]:
df.head()

,type,price,addition
0,букет,3000,1
1,композиция,1500,0
2,один цветок,200,1
3,композиция,2000,1
4,один цветок,300,0


In [295]:
class NaiveBayes:
    def __init__(self, data, classes = None, target = None, a = None, type_f = 'classic'):
        self.data = data.copy()
        self.target = target
        if classes:
            self.classes = classes
        else:
            self.classes = df.iloc[:, -1].unique()
        self.a = a
        self.type_f = type_f
        self.predict_proba = []
    
    def proba(self, record):
        prob = []
        
        for c in self.classes:
            prob_a_b = 1
            all_n = self.data[self.data[self.target] == c][self.data.columns[0]].count()
            for i in range(len(record)-1):
                prob_a = all_n / self.data.shape[0]
                prob_b = self.data[(self.data[self.data.columns[i]] == record[i])][self.data.columns[0]].count() / self.data.shape[0]
                prob_b_a = self.data[(self.data[self.data.columns[i]] == record[i]) &\
                                               (self.data[self.target] == c)][self.data.columns[0]].count() / all_n
                prob_a_b *= (prob_b_a * prob_a / prob_b)
                       
            prob.append(prob_a_b)
            
        self.predict_proba.append(np.array(prob)/np.array(prob).sum()) 
        return self.classes[prob.index(max(prob))]
    
    def proba_smooth(self, record):
        prob = []
        
        for c in self.classes:
            prob_a_b = 1
            all_n = self.data[self.data[self.target] == c][self.data.columns[0]].count()
            for i in range(len(record)-1):
                prob_a = all_n / self.data.shape[0]
                prob_b = self.data[(self.data[self.data.columns[i]] == record[i])][self.data.columns[0]].count()
                prob_b_a = (self.data[(self.data[self.data.columns[i]] == record[i]) &\
                                    (self.data[self.target] == c)][self.data.columns[0]].count() + self.a) / (all_n + self.a)

                prob_a_b *= (prob_b_a * prob_a / prob_b)
                       
            prob.append(prob_a_b)
        self.predict_proba.append(np.array(prob)/np.array(prob).sum())    
        return self.classes[prob.index(max(prob))]
    
    def fit(self):
        if self.target is None:
            self.target = df.columns[-1]
        if self.type_f == 'classic' and self.a is None:
            self.data['predict'] = self.data.apply(lambda x: self.proba(x.values.flatten().tolist()), axis=1) 
        else:
            self.data['predict'] = self.data.apply(lambda x: self.proba_smooth(x.values.flatten().tolist()), axis=1) 
        return self.data
        

In [299]:
naiv = NaiveBayes(df)
naiv.fit()

,type,price,addition,predict
0,букет,3000,1,1
1,композиция,1500,0,1
2,один цветок,200,1,0
3,композиция,2000,1,1
4,один цветок,300,0,0
5,несколько цветков,1500,1,1
6,букет,5000,0,0
7,композиция,2000,1,1
8,один цветок,200,0,0


In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y, np.array(naiv.predict_proba)[:, 0])

In [308]:
naiv_smooth = NaiveBayes(df, a = 1)
naiv_smooth.fit()

,type,price,addition,predict
0,букет,3000,1,1
1,композиция,1500,0,1
2,один цветок,200,1,0
3,композиция,2000,1,1
4,один цветок,300,0,0
5,несколько цветков,1500,1,1
6,букет,5000,0,0
7,композиция,2000,1,1
8,один цветок,200,0,0


In [309]:
roc_auc_score(y, np.array(naiv.predict_proba)[:, 0])

0.925